In [1]:
import pandas as pd
import os
from datetime import datetime
from datetime import date
import numpy as np
daily_direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/BI Argentina/Daily Táctico'
repos_direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/Bajadas_repo_ventas'
lista_archivos = [x for x in os.listdir(repos_direc) if len(x)==44]

#ver en el OI a que columna corresponde este mes
col_excel_u = 'DQ:DS'
col_excel_vn = 'JX:JZ'

#aca pongo hasta que mes junto todos los archivos en una tabla
procesar_hasta = '2023_01'

#Leo el archivo de materiales
materiales = pd.read_excel(io = "C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/Tbl_maestro_articulos.xlsx", usecols="D:J,L")

In [2]:
materiales

,Cod_Material,Material,Empresa,Status,BU,Categoría,Marca,Familia
0,8.100000e+09,CICATRICURE CONTORNO OJOS 8.5G LATAM,GLA,DISCO,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE CONTORNO DE OJOS
1,8.100002e+09,ASX JBN FOR CON ZINC SEBUM X 100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
2,8.100002e+09,ASX JBN HERBAL CON ZINC SEBUM X 100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
3,8.100002e+09,ASX JBN AZF C/ZINC SEBUM X100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
4,8.100002e+09,ASX JBN HUM CON ZINC SEBUM X 100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
...,...,...,...,...,...,...,...,...
919,8.100005e+09,ASPX PIÑA POLVO LIMPIADOR 42G /12,GLA,INNO 1,PC,SKIN CARE,ASEPXIA,ASEPXIA PIÑA
920,8.110001e+09,TÍO NACHO CRECIMIENTO SALUDABLE CREMA PARA PEI...,GLA,INNO 1,PC,HAIR CARE,TIO NACHO,TIO NACHO CRECIMIENTO SALUDABLE
921,8.110001e+09,CICATRICURE READY TO SELL EYE CR FACE,GLA,INNO 1,PC,SKIN CARE,CICATRICURE FACIAL,EYECREAMFORFACE RTS
922,8.110001e+09,BENGUE SIN OLOR ROLL ON 50G,GLA,INNO 1,OTC,ANALGESICS,BENGUE,BENGUE SIN OLOR


In [3]:
def procesar_repos(file):
    
    #nombres de las columnas del archivo de SAP
    SAP_names = ['Doc.fact.', 'Tipo Doc.', 'Folio Entrega', 'Documento de ventas', 'Nombre' ,
                 'Destinatario', 'Material', 'Cantidad facturada', 'Valor neto', 'Importe Bruto', 'Descuentos', 
                 'Fecha factura', 'Canal distribución','Organización ventas', 'Clase de factura']

    #nombres de las columnas de BD_Venta
    BD_names = ['Nro_factura', 'Tipo_doc', 'Nro_entrega', 'Nro_pedido', 'Nbre_Solicitante', 
                'Nro_Destinatario', 'Cod_Material', 'Unidades', 'Valor_Neto', 'Venta_Bruta', 'Descuentos', 
                'Fe_factura', 'Canal_Distribucion', 'Organizacion ventas', 'Clase_de_factura', 'Año_Mes']
    
    #Leo el archivo y filtro las columnas
    x = pd.read_excel(repos_direc + '/' + file)    
    x = x[SAP_names]
    
    #Agrego la columna Año_Mes usando el nombre del archivo
    x['Año_Mes'] = file[-12:-5].replace('_','-')
    
    #Filtro filas, El material 9100000322 lo pongo cuando tenga los montos
    x = x[x['Organización ventas']=='VNAR']
    x = x[~x['Material'].isin([9100000322, 9100000316])]
    
    x.drop('Organización ventas', axis=1, inplace=True) #elimino la columna 'Organización ventas'
    
    x['Tipo'] = 'real'
    
    x.rename(dict(zip(SAP_names,BD_names)), axis=1, inplace=True) #cambio los nombres de las columnas

    x['Unidades'] = x['Unidades'].astype('int64')

    dict_replace = {"NC Sin Stock AR" : "NC",
                    "Nota de Crédito AR" : "NC",
                    "NC Recupero AR" : "NC",
                    "Factura Nacional AR" : "Fact",
                    "Factura Servicios AR" : "Fact",
                    "Nota de Debito AR" : "ND"}

    x = x.replace({"Tipo_doc": dict_replace})

    x = x[x['Tipo_doc'].isin(['NC', 'Fact', 'ND'])]

    #Me creo el campo Venta Neta
    x['Valor_Back'] = x['Valor_Neto']
    x.loc[x['Clase_de_factura']!='ZNCS', 'Valor_Back'] = 0
    x.loc[x['Tipo_doc'].isin(['NC', 'ND']), 'Venta_Bruta'] = -x['Venta_Bruta']
    
    return x

In [4]:
%%time

procesar_hasta = [i for i in range(0,len(lista_archivos)) if lista_archivos[i][32:39] == procesar_hasta][0]

repos_ventas = pd.read_csv(daily_direc + '/REPOS_VENTA_procesado/repos.csv', sep = ';')

if (lista_archivos[procesar_hasta][32:39].replace('_','-') != repos_ventas.Año_Mes.max()):
    
    #si la fecha hasta la que debo procesar no es la misma que la fecha maxima que tengo,
    #agrego a la base los archivos que faltan y los sobreescribo
    repos_ventas = pd.DataFrame()
    for file in lista_archivos[:procesar_hasta+1]:
        repos_ventas = pd.concat([repos_ventas, procesar_repos(file)], ignore_index=True)
        
    repos_ventas.to_csv(path_or_buf = daily_direc + '/REPOS_VENTA_procesado/repos.csv',
                    sep = ';', index = False)

#luego concateno los archivos que faltan
for file in lista_archivos[procesar_hasta+1:]:
    repos_ventas = pd.concat([repos_ventas, procesar_repos(file)], ignore_index=True)
    
repos_ventas['Fe_factura'] = pd.to_datetime(repos_ventas['Fe_factura'])

Wall time: 10.9 s


In [5]:
repos_ventas

,Nro_factura,Tipo_doc,Nro_entrega,Nro_pedido,Nbre_Solicitante,Nro_Destinatario,Cod_Material,Unidades,Valor_Neto,Venta_Bruta,Descuentos,Fe_factura,Canal_Distribucion,Clase_de_factura,Año_Mes,Tipo,Valor_Back
0,2000087316,Fact,170088514.0,170088514,ASOPROFARMA ASOC. PROP.,941,8.110000e+09,560,240561.78,264353.60,-23791.82,2021-01-04,MD,ZFCE,2021-01,real,0.0
1,2000087317,Fact,180522670.0,170088496,DIA ARGENTINA SA,1899,8.110000e+09,10500,1215043.20,1852200.00,-637156.80,2021-01-04,A1,ZFCE,2021-01,real,0.0
2,2000087317,Fact,180522670.0,170088496,DIA ARGENTINA SA,1899,8.110000e+09,10500,1215043.20,1852200.00,-637156.80,2021-01-04,A1,ZFCE,2021-01,real,0.0
3,2000087318,Fact,180522671.0,170088498,ALBERDI S.A.,760,8.110000e+09,180,39550.50,42075.00,-2524.50,2021-01-04,A2,ZFCE,2021-01,real,0.0
4,2000087318,Fact,180522671.0,170088498,ALBERDI S.A.,760,8.110000e+09,264,58007.40,61710.00,-3702.60,2021-01-04,A2,ZFCE,2021-01,real,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713272,2000164948,Fact,180724406.0,170169355,ENRIQUE REYES S.A.,997,8.110000e+09,100,107842.44,114726.00,-6883.56,2023-03-08,DI,ZFCE,2023-03,real,0.0
713273,2000164949,Fact,180724407.0,170169377,MARCAS SRL,1100978,8.110000e+09,40,43595.88,45890.40,-2294.52,2023-03-08,DI,ZFCE,2023-03,real,0.0
713274,2000164949,Fact,180724407.0,170169377,MARCAS SRL,1100978,8.110001e+09,12,23050.57,24263.76,-1213.19,2023-03-08,DI,ZFCE,2023-03,real,0.0
713275,2000164950,Fact,180724408.0,170169472,ALEJANDRO JOSE GRAY,2109,8.110000e+09,60,57108.82,62074.80,-4965.98,2023-03-08,DI,ZFCE,2023-03,real,0.0


In [6]:
repos_ventas.groupby(['Año_Mes'])['Unidades'].sum()

Año_Mes
2021-01    2897957
2021-02    2889110
2021-03    4451678
2021-04    3775889
2021-05    3255309
2021-06    3992864
2021-07    3658815
2021-08    4265437
2021-09    4264081
2021-10    3576568
2021-11    3491856
2021-12    3929269
2022-01    4773788
2022-02    6059205
2022-03    6692134
2022-04    6899571
2022-05    6719529
2022-06    6013681
2022-07    6211162
2022-08    6077477
2022-09    5189184
2022-10    4984078
2022-11    6006375
2022-12    4524575
2023-01    5684471
2023-02    5656926
2023-03    1898243
Name: Unidades, dtype: int64

In [7]:
#Son los materiales que estan en repo ventas que no estan en el maestro. El vector deberia tener un len de cero
pd.unique(repos_ventas[~repos_ventas['Cod_Material'].isin(materiales['Cod_Material'])]['Cod_Material']).astype('int64')

array([8110000655], dtype=int64)

In [8]:
repos2 = pd.merge(repos_ventas,
                materiales,
                left_on = 'Cod_Material',
                right_on = 'Cod_Material',
                how = 'left')

In [9]:
def read_oi(col_excel, col_name):
    
    oi = pd.read_excel(daily_direc + '/OI_CANAL/OI 2023-02 009 Cerrado.XLSX', header = 6, sheet_name = 'Rolling Forecast', 
                   usecols = 'L,' + col_excel)
    
    oi.drop(oi.tail(9).index,inplace=True) # drop last 4 rows 

    try:
        fechas = [x.strftime('%Y-%m') for x in oi.columns[1:]] # me creo el vector de fechas limpias
    except:
        oi.rename(dict(zip(oi.columns.to_list(),[x.split('.')[0] for x in oi.columns])), axis=1, inplace=True)
        #como me queda en formato string, lo convierto a date
        fechas = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m') for x in oi.columns[1:]]
    else:
        fechas = [x.strftime('%Y-%m') for x in oi.columns[1:]] # me creo el vector de fechas limpias

    #renombro columnas
    oi_names = ['Cod_Material'] + fechas
    oi.rename(dict(zip(oi.columns.to_list(),oi_names)), axis=1, inplace=True)
    
    oi = pd.melt(oi, 
             id_vars = 'Cod_Material',
             value_vars = fechas,
             value_name = col_name, var_name = 'Año_Mes')

    #convierto unidades a int
    oi[col_name] = oi[col_name].fillna(0)
    oi[col_name] = oi[col_name].astype('int64')
    oi = oi.dropna(subset=['Cod_Material'])
    oi['Cod_Material'] = oi['Cod_Material'].astype('int64')
    
    #agrego la columna de tipo 
    oi['Tipo'] = 'proy'

    return oi

In [10]:
%%time

oi = pd.merge(read_oi(col_excel_u, 'Unidades'),
                read_oi(col_excel_vn, 'Valor_Neto'),
                left_on = ['Cod_Material', 'Tipo', 'Año_Mes'],
                right_on = ['Cod_Material', 'Tipo', 'Año_Mes'],
                how = 'left')
oi.drop_duplicates(inplace = True)

oi

C:\Users\rvozzi\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


Wall time: 8.68 s


,Cod_Material,Año_Mes,Unidades,Tipo,Valor_Neto
0,8110000329,2023-01,0,proy,0
1,8110000013,2023-01,0,proy,0
2,8110000012,2023-01,0,proy,0
3,8100003447,2023-01,0,proy,0
4,8100003126,2023-01,0,proy,0
...,...,...,...,...,...
6133,8110000895,2023-03,0,proy,0
6134,8110000896,2023-03,0,proy,0
6135,8110000897,2023-03,0,proy,0
6136,8110000775,2023-03,0,proy,0


In [11]:
#Chequeo contra el subtotal del OI de Unidades
oi.groupby(['Año_Mes'])['Unidades'].sum()

Año_Mes
2023-01    5724325
2023-02    5697090
2023-03    7234731
Name: Unidades, dtype: int64

In [12]:
#Chequeo contra el subtotal del OI de Valor Neto
oi.groupby(['Año_Mes'])['Valor_Neto'].sum()

Año_Mes
2023-01    2973855685
2023-02    3124472435
2023-03    4934616832
Name: Valor_Neto, dtype: int64

In [13]:
oi2 = pd.merge(oi, materiales, left_on = 'Cod_Material',
                right_on = 'Cod_Material',
                how = 'inner')

In [14]:
def calc_fechas(fecha):
    
    fechas = pd.date_range(fecha[5:7] + '-' +'01-' +  str(int(fecha[2:4])-1),fecha[5:7] + '-' + '01-' + fecha[2:4], 
                         freq='MS').strftime("%Y-%m").tolist()
    
    return fechas[:-1]

In [15]:
def calcular_factor(valor):
    
    #calculo el total del periodo por descripcion, por cliente
    cli_per = repos2[repos2['Año_Mes'].isin(calc_fechas(pd.unique(oi_iter['Año_Mes'])[0]))]
    cli_per = cli_per.groupby(['Cod_Material', 'Nbre_Solicitante'],
                        as_index = False)[valor].sum()
    
    #calculo el total del periodo por descripcion
    desc_per = repos2[repos2['Año_Mes'].isin(calc_fechas(pd.unique(oi_iter['Año_Mes'])[0]))]
    desc_per = desc_per.groupby(['Cod_Material'],
                    as_index = False)[valor].sum()
    name2 = 'Total' + '_' + valor
    desc_per.rename(columns={valor: name2}, inplace = True)
    
    #calculo los factores del periodo
    factores = pd.merge(cli_per,
                desc_per,
                left_on = ['Cod_Material'],
                right_on = ['Cod_Material'],
                how = 'inner')
    name3 = 'factor' + '_' + valor
    factores[name3] = factores[valor] / factores[name2]
    factores.replace([np.inf, -np.inf], 0, inplace=True)
    factores.fillna(value = 0, inplace = True)
    return factores

In [16]:
factores = pd.DataFrame()

for mes in pd.unique(oi['Año_Mes']):
    
    oi_iter = oi[oi['Año_Mes'].isin([mes])]

    factores_u = calcular_factor('Unidades')
    factores_u.fillna(value = 0, inplace = True)

    factores_v = calcular_factor('Valor_Neto')
    factores_v.fillna(value = 0, inplace = True)

    x = pd.merge(factores_u,
                       factores_v,
                       left_on = ['Cod_Material', 'Nbre_Solicitante'],
                       right_on = ['Cod_Material', 'Nbre_Solicitante'],
                       how = 'inner')
    
    x['Año_Mes'] = mes
    
    factores = pd.concat([factores, x], ignore_index=True)

factores

,Cod_Material,Nbre_Solicitante,Unidades,Total_Unidades,factor_Unidades,Valor_Neto,Total_Valor_Neto,factor_Valor_Neto,Año_Mes
0,8.100000e+09,ACOFAR COOPERATIVA,553,34373,0.016088,561638.30,28399688.63,0.019776,2023-01
1,8.100000e+09,ALBERDI S.A.,10,34373,0.000291,3453.07,28399688.63,0.000122,2023-01
2,8.100000e+09,ALEJANDRO JOSE GRAY,47,34373,0.001367,27039.30,28399688.63,0.000952,2023-01
3,8.100000e+09,ASOPROFARMA ASOC. PROP.,1067,34373,0.031042,982690.14,28399688.63,0.034602,2023-01
4,8.100000e+09,AUTOSERVICIO MAYORISTA,84,34373,0.002444,59222.89,28399688.63,0.002085,2023-01
...,...,...,...,...,...,...,...,...,...
62248,9.100001e+09,MODENA DISTRIBUCIONES SRL,-1,-43,0.023256,-33750.84,-10627838.19,0.003176,2023-03
62249,9.100001e+09,RAFCA SRL,-1,-43,0.023256,-244106.74,-10627838.19,0.022969,2023-03
62250,9.100001e+09,SUPERMERCADO MAY YAGUAR SA,-1,-43,0.023256,-105628.36,-10627838.19,0.009939,2023-03
62251,9.100001e+09,TADICOR SA,-2,-43,0.046512,-58140.40,-10627838.19,0.005471,2023-03


In [17]:
factores['factor_Unidades'].sum()

1501.9999999999998

In [18]:
factores['factor_Valor_Neto'].sum()

1502.0

In [19]:
len(pd.unique(repos_ventas['Cod_Material']))

607

In [20]:
# para cada uno de los meses del OI quiero reemplazar los factores inexistentes 
# de los codigos nuevos para aquel momento, por otros codigos primos

i = 0

agregar_dict = {}
meses_dict = {}

for mes in pd.unique(oi['Año_Mes']):

    res = pd.pivot_table(repos2[repos2['Año_Mes'].isin(calc_fechas(mes))], 
                         values='Unidades', index=['Cod_Material', 'Material'],
                         #columns='Año_Mes', 
                         aggfunc=np.sum)

    res = res.reset_index()

    oi2_iter = oi2[oi2['Año_Mes'].isin([mes])]
    
    agregar_dict[i] = pd.unique(oi2_iter[oi2_iter['Unidades']>0][~oi2_iter[oi2_iter['Unidades']>0]['Cod_Material'].isin(res['Cod_Material'])]['Cod_Material'])
    
    meses_dict[i] = mes
    
    i = i + 1
    

print(agregar_dict)
print(meses_dict)

{0: array([8110000724, 8110000684, 8110000882, 8110000875, 8110000877,
       8110000880], dtype=int64), 1: array([8100005159, 8100005383, 8100005162, 8110000883, 8110000850],
      dtype=int64), 2: array([8110000819, 8110000881], dtype=int64)}
{0: '2023-01', 1: '2023-02', 2: '2023-03'}


In [21]:
#recordar que ahora la tabla de factores tiene el Año_Mes
#tengo que chequear que el valor del diccionario en el mismo indice sea de la misma longitud
#luego modificar los factores para el mes correspondiente en el data frame

agregar_val = {0: [8110000863, 8110000096, 8110000129, 8110000863, 8110000865, 8110000117],
               1: [8100005576, 8100005576, 8100005576, 8110000780, 8110000129],
               2: [8110000537, 8110000712]}

i = 0

for mes in pd.unique(oi['Año_Mes']):

    agregar_factores = pd.DataFrame()

    if (len(agregar_dict.get(i))==len(agregar_val.get(i))):
        #hago un replace de la tabla factores
        #me cargo los df a agregar_factores
        for k in range(0,len(agregar_dict.get(i))):

            x = factores[factores['Cod_Material']==agregar_val.get(i)[k]]
            x = x[x['Año_Mes']==mes]
            x = x.replace(agregar_val.get(i)[k], agregar_dict.get(i)[k])

            agregar_factores = pd.concat([agregar_factores, x], ignore_index=True)

    else:
        print("la lista agregar val debe ser modificada")

    factores = pd.concat([factores, agregar_factores], ignore_index=True)
    
    i = i + 1

In [22]:
factores

,Cod_Material,Nbre_Solicitante,Unidades,Total_Unidades,factor_Unidades,Valor_Neto,Total_Valor_Neto,factor_Valor_Neto,Año_Mes
0,8.100000e+09,ACOFAR COOPERATIVA,553,34373,0.016088,561638.30,28399688.63,0.019776,2023-01
1,8.100000e+09,ALBERDI S.A.,10,34373,0.000291,3453.07,28399688.63,0.000122,2023-01
2,8.100000e+09,ALEJANDRO JOSE GRAY,47,34373,0.001367,27039.30,28399688.63,0.000952,2023-01
3,8.100000e+09,ASOPROFARMA ASOC. PROP.,1067,34373,0.031042,982690.14,28399688.63,0.034602,2023-01
4,8.100000e+09,AUTOSERVICIO MAYORISTA,84,34373,0.002444,59222.89,28399688.63,0.002085,2023-01
...,...,...,...,...,...,...,...,...,...
63126,8.110001e+09,SUIZO ARGENTINA S.A.,5144,74812,0.068759,5753266.03,80870631.74,0.071142,2023-03
63127,8.110001e+09,TREOLAND S.A.,773,74812,0.010333,679920.91,80870631.74,0.008408,2023-03
63128,8.110001e+09,UOM BUSTAMANTE S.C.S.,5,74812,0.000067,10165.30,80870631.74,0.000126,2023-03
63129,8.110001e+09,UOM DELIA SCS,28,74812,0.000374,46172.95,80870631.74,0.000571,2023-03


In [23]:
tabla = pd.merge(factores[[ 'Cod_Material', 'Nbre_Solicitante', 'factor_Unidades', 'factor_Valor_Neto', 'Año_Mes']], 
                 oi2, 
                 left_on = ['Cod_Material', 'Año_Mes'],
                 right_on = ['Cod_Material', 'Año_Mes'],
                 how = 'inner')
tabla.rename(columns={"Unidades": "Total"}, inplace = True)
tabla.rename(columns={"Valor_Neto": "Total_VN"}, inplace = True)

tabla['Unidades'] = tabla['factor_Unidades']*tabla['Total']
tabla['Valor_Neto'] = tabla['factor_Valor_Neto']*tabla['Total_VN']

tabla

,Cod_Material,Nbre_Solicitante,factor_Unidades,factor_Valor_Neto,Año_Mes,Total,Tipo,Total_VN,Material,Empresa,Status,BU,Categoría,Marca,Familia,Unidades,Valor_Neto
0,8.100000e+09,ACOFAR COOPERATIVA,0.016088,0.019776,2023-01,0,proy,-24463,CICATRICURE CONTORNO OJOS 8.5G LATAM,GLA,DISCO,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE CONTORNO DE OJOS,0.000000,-4.837855e+02
1,8.100000e+09,ALBERDI S.A.,0.000291,0.000122,2023-01,0,proy,-24463,CICATRICURE CONTORNO OJOS 8.5G LATAM,GLA,DISCO,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE CONTORNO DE OJOS,0.000000,-2.974415e+00
2,8.100000e+09,ALEJANDRO JOSE GRAY,0.001367,0.000952,2023-01,0,proy,-24463,CICATRICURE CONTORNO OJOS 8.5G LATAM,GLA,DISCO,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE CONTORNO DE OJOS,0.000000,-2.329118e+01
3,8.100000e+09,ASOPROFARMA ASOC. PROP.,0.031042,0.034602,2023-01,0,proy,-24463,CICATRICURE CONTORNO OJOS 8.5G LATAM,GLA,DISCO,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE CONTORNO DE OJOS,0.000000,-8.464723e+02
4,8.100000e+09,AUTOSERVICIO MAYORISTA,0.002444,0.002085,2023-01,0,proy,-24463,CICATRICURE CONTORNO OJOS 8.5G LATAM,GLA,DISCO,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE CONTORNO DE OJOS,0.000000,-5.101357e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62839,8.110001e+09,SUIZO ARGENTINA S.A.,0.068759,0.071142,2023-03,11822,proy,21761986,CICATRICURE SERUM ACLARANTE VITC AR 30ML,GLA,NaN,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,812.869165,1.548182e+06
62840,8.110001e+09,TREOLAND S.A.,0.010333,0.008408,2023-03,11822,proy,21761986,CICATRICURE SERUM ACLARANTE VITC AR 30ML,GLA,NaN,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,122.151607,1.829642e+05
62841,8.110001e+09,UOM BUSTAMANTE S.C.S.,0.000067,0.000126,2023-03,11822,proy,21761986,CICATRICURE SERUM ACLARANTE VITC AR 30ML,GLA,NaN,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,0.790114,2.735444e+03
62842,8.110001e+09,UOM DELIA SCS,0.000374,0.000571,2023-03,11822,proy,21761986,CICATRICURE SERUM ACLARANTE VITC AR 30ML,GLA,NaN,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,4.424638,1.242497e+04


In [24]:
for ma in pd.unique(oi2['Cod_Material']):
    if round((tabla[tabla['Cod_Material']==ma]['Unidades'].sum()-oi2[oi2['Cod_Material']==ma]['Unidades'].sum()),0)!=0:
        print(ma)
        print(tabla[tabla['Cod_Material']==ma]['Unidades'].sum()-oi2[oi2['Cod_Material']==ma]['Unidades'].sum())

In [25]:
tabla.groupby(['Año_Mes'])['Unidades'].sum()

Año_Mes
2023-01    5724325.0
2023-02    5697090.0
2023-03    7024731.0
Name: Unidades, dtype: float64

In [26]:
oi.groupby(['Año_Mes'])['Unidades'].sum()

Año_Mes
2023-01    5724325
2023-02    5697090
2023-03    7234731
Name: Unidades, dtype: int64

In [27]:
tabla.groupby(['Año_Mes'])['Valor_Neto'].sum()

Año_Mes
2023-01    2.973856e+09
2023-02    3.124516e+09
2023-03    4.819669e+09
Name: Valor_Neto, dtype: float64

In [28]:
oi.groupby(['Año_Mes'])['Valor_Neto'].sum()

Año_Mes
2023-01    2973855685
2023-02    3124472435
2023-03    4934616832
Name: Valor_Neto, dtype: int64

In [29]:
tabla.columns

Index(['Cod_Material', 'Nbre_Solicitante', 'factor_Unidades',
       'factor_Valor_Neto', 'Año_Mes', 'Total', 'Tipo', 'Total_VN', 'Material',
       'Empresa', 'Status', 'BU', 'Categoría', 'Marca', 'Familia', 'Unidades',
       'Valor_Neto'],
      dtype='object')

In [30]:
oi.columns

Index(['Cod_Material', 'Año_Mes', 'Unidades', 'Tipo', 'Valor_Neto'], dtype='object')

In [31]:
oi = tabla[['Cod_Material', 'Nbre_Solicitante','Unidades', 'Valor_Neto', 'Tipo', 'Año_Mes']]

In [32]:
#a estos productos los tengo que agregar a la tabla de materiales
a = oi[~oi['Cod_Material'].isin(materiales['Cod_Material'])]
a[a['Unidades']>0]

,Cod_Material,Nbre_Solicitante,Unidades,Valor_Neto,Tipo,Año_Mes


In [33]:
base = pd.concat([repos_ventas, oi])

base = pd.merge(base,
                materiales,
                left_on = 'Cod_Material',
                right_on = 'Cod_Material',
                how = 'left')

#junto la base con el catalogo de clientes
catalogo_clientes = pd.read_excel(io = "C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/TBL_canales_SAP_y_gerencias_ventas vR2D2.xlsx")

catalogo_clientes = catalogo_clientes[['Nbre_solicitante', 'Nbre_solicitante2',
       'Canal_distribucion', 'Tipo_estructura5', 'Gerente', 'KAM']]
catalogo_clientes.rename(columns={"Tipo_estructura5": "Canal"}, inplace = True)

base = pd.merge(base,
                catalogo_clientes,
                left_on = 'Nbre_Solicitante',
                right_on = 'Nbre_solicitante',
                how = 'left')

base.loc[base['Nbre_solicitante2'] != 'OTROS', 'Nbre_solicitante'] = base['Nbre_solicitante2']

base.drop(['Nbre_solicitante2'], axis=1, inplace=True)
base.drop(['Nbre_Solicitante'], axis=1, inplace=True)

base['Nro_factura'] = base['Nro_factura'].fillna(0)
base['Nro_factura'] = base['Nro_factura'].astype('int64')

base['Nro_entrega'] = base['Nro_entrega'].fillna(0)
base['Nro_entrega'] = base['Nro_entrega'].astype('int64')

base['Nro_pedido'] = base['Nro_pedido'].fillna(0)
base['Nro_pedido'] = base['Nro_pedido'].astype('int64')

base['Nro_Destinatario'] = base['Nro_Destinatario'].fillna(0)
base['Nro_Destinatario'] = base['Nro_Destinatario'].astype('int64')

base['Valor_Neto'] = base['Valor_Neto'].fillna(0)

base.rename(columns={"Material": "Descripcion"}, inplace = True)

#junto la base con el archivo de ship to x clientes
ship = pd.read_excel('C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/20221018 Detalle de Ship tos X Clientes ARG.XLSX', usecols='AN,AT')
ship.columns = ['ship to', 'ship to poblacion']
ship = ship.drop_duplicates()

base = pd.merge(base,
                ship,
                left_on = 'Nro_Destinatario',
                right_on = 'ship to',
                how = 'left')

In [34]:
base[base['Nbre_solicitante'].isnull()]

,Nro_factura,Tipo_doc,Nro_entrega,Nro_pedido,Nro_Destinatario,Cod_Material,Unidades,Valor_Neto,Venta_Bruta,Descuentos,...,Categoría,Marca,Familia,Nbre_solicitante,Canal_distribucion,Canal,Gerente,KAM,ship to,ship to poblacion
871,2000087363,Fact,180522678,170088543,1901,8.110000e+09,36.000000,6787.990000,10892.15,-4104.16,...,HAIR CARE,TIO NACHO,TIO NACHO ENGROSADOR,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
872,2000087363,Fact,180522678,170088543,1901,8.110000e+09,42.000000,11878.990000,19061.28,-7182.29,...,HAIR CARE,TIO NACHO,TIO NACHO ENGROSADOR,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
873,2000087363,Fact,180522678,170088543,1901,8.110000e+09,120.000000,22626.640000,36307.19,-13680.55,...,HAIR CARE,TIO NACHO,TIO NACHO ULTRAHIDRATANTE,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
874,2000087363,Fact,180522678,170088543,1901,8.110000e+09,102.000000,28848.980000,46291.68,-17442.70,...,HAIR CARE,TIO NACHO,TIO NACHO ULTRAHIDRATANTE,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
875,2000087363,Fact,180522678,170088543,1901,8.110001e+09,6.000000,1697.000000,2723.04,-1026.04,...,HAIR CARE,TIO NACHO,TIO NACHO ALOE VERA,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776031,0,NaN,0,0,0,8.110001e+09,87.645243,70223.283429,NaN,NaN,...,ANTIMICOTIC,LOMECAN,TIO NACHO HERBOLARIA RE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776052,0,NaN,0,0,0,8.110001e+09,3.476501,10810.466668,NaN,NaN,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776065,0,NaN,0,0,0,8.110001e+09,24.335508,34995.804113,NaN,NaN,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776086,0,NaN,0,0,0,8.110001e+09,6.320911,2335.793366,NaN,NaN,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
base.to_csv(path_or_buf = daily_direc + '/base_final.csv',
                    sep = ';', index = False)

In [36]:
base

,Nro_factura,Tipo_doc,Nro_entrega,Nro_pedido,Nro_Destinatario,Cod_Material,Unidades,Valor_Neto,Venta_Bruta,Descuentos,...,Categoría,Marca,Familia,Nbre_solicitante,Canal_distribucion,Canal,Gerente,KAM,ship to,ship to poblacion
0,2000087316,Fact,170088514,170088514,941,8.110000e+09,560.000000,2.405618e+05,264353.6,-23791.82,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE BEAUTY CARE,ASOPROFARMA ASOC. PROP.,MD,1.Farma,Catalina Sabogal,Sandra Borguetto,941.0,CARAPACHAY
1,2000087317,Fact,180522670,170088496,1899,8.110000e+09,10500.000000,1.215043e+06,1852200.0,-637156.80,...,HAIR CARE,TIO NACHO,TIO NACHO HERBOLARIA,DIA ARGENTINA SA,A1,3.Supermercados,Vacante,Nicolas Sardi,1899.0,TORTUGUITAS
2,2000087317,Fact,180522670,170088496,1899,8.110000e+09,10500.000000,1.215043e+06,1852200.0,-637156.80,...,HAIR CARE,TIO NACHO,TIO NACHO HERBOLARIA,DIA ARGENTINA SA,A1,3.Supermercados,Vacante,Nicolas Sardi,1899.0,TORTUGUITAS
3,2000087318,Fact,180522671,170088498,760,8.110000e+09,180.000000,3.955050e+04,42075.0,-2524.50,...,HAIR CARE,TIO NACHO,TIO NACHO HERBOLARIA,COMODIN,A2,3.Supermercados,Vacante,Sebastian Beltran,760.0,PALPALÁ
4,2000087318,Fact,180522671,170088498,760,8.110000e+09,264.000000,5.800740e+04,61710.0,-3702.60,...,HAIR CARE,TIO NACHO,TIO NACHO ACLARANTE,COMODIN,A2,3.Supermercados,Vacante,Sebastian Beltran,760.0,PALPALÁ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776116,0,NaN,0,0,0,8.110001e+09,812.869165,1.548182e+06,NaN,NaN,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,SUIZO ARGENTINA S.A.,MD,1.Farma,Catalina Sabogal,Rodrigo Gonzalez Calderón,NaN,NaN
776117,0,NaN,0,0,0,8.110001e+09,122.151607,1.829642e+05,NaN,NaN,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,TORNADO,DI,5.Tradicional,Diego Dopazo,Diego Dopazo,NaN,NaN
776118,0,NaN,0,0,0,8.110001e+09,0.790114,2.735444e+03,NaN,NaN,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,UOM BUSTAMANTE S.C.S.,FR,2.Cadenas farmacias,Catalina Sabogal,Analia Garcia,NaN,NaN
776119,0,NaN,0,0,0,8.110001e+09,4.424638,1.242497e+04,NaN,NaN,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,UOM DELIA SCS,FR,2.Cadenas farmacias,Catalina Sabogal,Analia Garcia,NaN,NaN


In [37]:
factores.to_excel('C:/Users/rvozzi/Downloads/factores.xlsx')